# Multi Module External Trigger Test

# Test Description
This script will test the Multimodule External triggering capability of proteus. In this all the channels of all the modules are getting triggered from external signal generator source frequency though external signal generator is connected to TRG1 or TRG2 input of Master module only.This script test for both triggers defined TRG1 and TRG2. This test will run for multiple sampling rate from 1GHz to 2.5GHz for all channels and from 2.5GHz to 9GHz for Channel 1 & 3 only.

# Test Equipment
1. Tabor Proteus 9484M
2. Keysight Oscilloscope MSO9254A
3. 4 SMA to BNC Cable
4. Tabor WW1072 waveform generator
5. 1 SMP to SMP Cable

## Tabor WW1072 waveform generator configuration
1. Wave shape - Square
2. Frequency - 100 KHz
3. Amplitude - 2 Vpp
4. Offset - 0


# Hardware Connection

    Proteus Master module REF OUT ---> Proteus Slave module REF IN 


    Proteus Module    --->     Oscilloscope
    Channel 1         --->      Channel 1
    Channel 2         --->      Channel 2
    Channel 3         --->      Channel 3
    Channel 4         --->      Channel 4
    
    
    Output of Signal generator ---> TRG1 or TRG2 of Proteus Module
         



# Test Procedure
1. Connect all the channels of Proteus module with the concerned channel of oscilloscope.
2. Connect external signal generator with TRG1 or TRG2 input.
3. Run the script and observe output on scope.

In [1]:
# Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
#matplotlib notebook
import pyvisa as visa
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from scipy import signal
import math
import pdb

In [2]:
## Connect to instrument
connection = 'LAN'
if connection == 'PXI':   
    # not setting this command means DLL are taken from SYS32
    #set_lib_dir_path(r'D:\Projects\ProteusAwg_Anabelle\x64\Debug') 
    # for service connection enter instrument IP
    # for PXI DLL connection enter the module slot number with Auto=False
    # when Auto=True the lowest module will be connected
    inst=connect("6",Auto=False)
    
elif connection == 'LAN':
    proteus_addr = 'TCPIP::192.90.70.22::5025::SOCKET'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'USB':
    proteus_addr = 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'LOCAL':
    proteus_addr = 'TCPIP::127.0.0.1::5025::SOCKET'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
else:
    print("Please select the conection method first")
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)
inst.default_paranoia_level = 2

Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 


In [3]:
# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get model dependant parameters:
if model_name.startswith('P948'):
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 1
elif model_name.startswith('P908'):
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    offset_factor = 1
else:
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 2
    
half_dac = max_dac / 2.0    
# Get the DAC mode (8 bits or 16 bits)

resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0': 
    dac_mode=16 
else: dac_mode=8
   
print('DAC {0} bits'.format(dac_mode))

Model: P9484M 
DAC 16 bits


In [4]:
# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
#inst.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':TRAC:DEL:ALL') # Delete all segments of the programmable channel's DDR.
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

0, no error
Number of channels: 4


In [5]:
# Clear and Reset the Modules after selection
inst.send_scpi_cmd(':INST:ACT:SEL 1')
resp = inst.send_scpi_query(":INST:ACT:SEL?")
print("Selected module is" + resp)
inst.send_scpi_cmd('*CLS; *RST')
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)

inst.send_scpi_cmd(':INST:ACT:SEL 2')
resp = inst.send_scpi_query(":INST:ACT:SEL?")
print("Selected module is" + resp)
inst.send_scpi_cmd('*CLS; *RST')
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)

Selected module is1
Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 
Selected module is2
Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 


In [6]:
# Clear and Reset the Modules after selection
inst.send_scpi_cmd(':INST:ACT:SEL 1')
resp = inst.send_scpi_query(":INST:ACT:SEL?")
print("Selected module is" + resp)

inst.send_scpi_cmd(':XINS:SYNC:FOLL 0')
resp = inst.send_scpi_query(":XINS:SYNC:FOLL?")
print("Selected module is" + resp)

inst.send_scpi_cmd(':XINS:SYNC:FOLL 1')
resp = inst.send_scpi_query(":XINS:SYNC:FOLL?")
print("Selected module is" + resp)

inst.send_scpi_cmd(':TRIG:COUPLE ON')
#inst.send_scpi_cmd(':TRIG:CPU:MODE LOCAL')

Selected module is1
Selected module is0
Selected module is1


0

In [7]:
scope_addr2=  'USB0::0x2A8D::0x900E::MY55490134::INSTR'

# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
    
    
    
## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST?')
scope.write('*IDN?')
idn = scope.read()
print('*IDN? returned: %s' % idn.rstrip('\n'))


*IDN? returned: KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101


In [ ]:
InitialFreqSampleRate = 1000
MaximumFreqSampleRate = 9000
FreqSampleRateJump = 1000

trigger_select = ['TRG1','TRG2']

    
for in_triggerselect in range(len(trigger_select)):
    print('***Testing start for Triggering input {}***\n'.format(trigger_select[in_triggerselect]))
    for i in range(2):
        inst_sel = i+1
        inst.send_scpi_cmd(':INST:ACT:SEL {}'.format(inst_sel))
        resp = inst.send_scpi_query(':SYST:ERR?')
#             print(resp)
        resp = inst.send_scpi_query(':INST:ACT:SEL?')
#             print('Instrument {} selceted'.format(resp))
        for FreqSampleRate in range (InitialFreqSampleRate,MaximumFreqSampleRate+FreqSampleRateJump,FreqSampleRateJump):
            if FreqSampleRate > 2500:
                channel_select = [1,3]
            else:
                channel_select = [1,2,3,4]
                    
            inst.send_scpi_cmd(':FREQ:RAST {} MHz'.format(int(FreqSampleRate)))
            FreqSampleRate = int(FreqSampleRate)

#             input("Press Enter")

            print('\t***Test begin for sampling rate {}MHz ***\n'.format(FreqSampleRate))    
        
            for in_channelselect in range(len(channel_select)):
                # Select Channel & Clear it for new operation
                inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))
                scope.write('*CLS;:DISPlay:CGRade:LEVels ')

                # Build waveforms on selected channel
                seglen = 2048
                ncycles = 1
                max_dac = 65535
                waves = [ None for _ in range(num_channels)]
                x = np.linspace(start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
                y = (np.sin(x)+1.0)*half_dac
                y = np.round(y)
                y = np.clip(y, 0, max_dac)
                if dac_mode == 16:
                    waves[0] = y.astype(np.uint16)
                else:
                    waves[0] = y.astype(np.uint8)
                #         plt.plot(x,y)
        #         print("************* WAVEFORM*****************")
                # Download segments
                for ii in range(1):
                    ichan = ii
                    channb = ichan + channel_select[in_channelselect]
                    segnum = 1
                    wav = waves[0]
        #             print('Download wave to segment {} of channel {}'.format(segnum, channb))
                    print('\t-->Test start for sampling rate {0}MHz for channel {1} for extnl trigg {2} for Module {3}'.format(FreqSampleRate,channel_select[in_channelselect],trigger_select[in_triggerselect],inst_sel))
                    # Select channel
                    inst.send_scpi_cmd(':INST:CHAN {}'.format(channb))
                    # Define segment
                    inst.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))
                    # Select the segment
                    inst.send_scpi_cmd(':TRAC:SEL {}'.format(segnum))
                    # Increase the timeout before writing binary-data:
                    inst.timeout = 30000
                    # Select the segment
                    cmd = ':TRAC:SEL {0}'.format(segnum)
                    inst.send_scpi_cmd(cmd)
                    # Send the binary-data:
                    inst.write_binary_data(':TRAC:DATA', wav)
                    resp = inst.send_scpi_query(':SYST:ERR?')
                    resp = resp.rstrip()
                    if not resp.startswith('0'):
                        print('ERROR: "{}" after writing binary values'.format(resp))
                    # Play the specified segment at the selected channel:
                    cmd = ':SOUR:FUNC:MODE:SEGM {}'.format(segnum)
                    inst.send_scpi_cmd(cmd)
                    # Turn on the output of the selected channel:
                    inst.send_scpi_cmd(':OUTP ON')
                    resp = inst.send_scpi_query(':SYST:ERR?')
#                     print(resp)
                           
#                 input("Press Enter")        
  
                              
                if(inst_sel==1):
                    scope.write('*CLS;:DISPlay:CGRade:LEVels ')
                    scope.write(':CHANnel{}:DISPlay ON'.format(channel_select[in_channelselect]))
                    scope.write(':CHANnel{}:PROBe 1.0'.format(channel_select[in_channelselect]))
                    scope.write(':CHANnel{}:SCALe 100E-3'.format(channel_select[in_channelselect]))
                    scope.write(':CHANnel{}:OFFSet 0.0'.format(channel_select[in_channelselect]))
                    scope.write(':TIMebase:SCALe 10E-6')
                    time.sleep(2)
                    scope.write(':MEASure:PERiod? CHANnel{}'.format(channel_select[in_channelselect]))
                    resualt = scope.read()
                    measured_period1 = float(resualt.split(',')[0])
#                     print(measured_period1)
                    scope.write(':MEASure:VPP CHANnel{}'.format(channel_select[in_channelselect]))
                    time.sleep(2)
                    scope.write(':MEASure:RESults?')
                    result = scope.read()
                    peak_peak_volt1= float(result.split(',')[1])
#                     print(peak_peak_volt1)
                    inst.send_scpi_cmd(':INIT:CONT OFF')
                    inst.send_scpi_cmd(':TRIG:LEV 0.2')
                    inst.send_scpi_cmd(':TRIG:COUPLE ON')
                    inst.send_scpi_cmd(':TRIG:CPU:MODE GLOBAL')
                    inst.send_scpi_cmd(':TRIG:SOUR:ENAB {}'.format(trigger_select[in_triggerselect]))
                    inst.send_scpi_cmd(':TRIG:SEL {}'.format(trigger_select[in_triggerselect]))
                    inst.send_scpi_cmd(':TRIG:STAT ON')
                    scope.write(':CHANnel{}:PROBe 1.0'.format(channel_select[in_channelselect]))
                    scope.write(':CHANnel{}:SCALe 100E-3'.format(channel_select[in_channelselect]))
                    scope.write(':CHANnel{}:OFFSet 0.0'.format(channel_select[in_channelselect]))
                    scope.write(':TIMebase:SCALe 10E-6')
                    scope.write(':TRIGger:MODE EDGE')
                    scope.write(':TRIGger:EDGE:SOURce CHANnel{}'.format(channel_select[in_channelselect]))
                    scope.write(':TRIGger:LEVel CHANnel{},120e-3'.format(channel_select[in_channelselect]))
                    scope.write(':MEASure:PERiod? CHANnel{}'.format(channel_select[in_channelselect]))
                    time.sleep(5)
                    resualt = scope.read()
                    measured_period2 = float(resualt.split(',')[0])
#                     print(measured_period2)
                    scope.write(':MEASure:VPP CHANnel{}'.format(channel_select[in_channelselect]))
                    scope.write(':MEASure:RESults?')
                    result = scope.read()
                    peak_peak_volt2= float(result.split(',')[1])
#                     print(peak_peak_volt2)
                    scope.write(':CHANnel{}:DISPlay OFF'.format(channel_select[in_channelselect]))
                    inst.send_scpi_cmd(':INIT:CONT ON')
                else:
                    scope.write('*CLS;:DISPlay:CGRade:LEVels ')
                    scope.write(':CHANnel{}:DISPlay ON'.format(channel_select[in_channelselect]))
                    scope.write(':CHANnel{}:PROBe 1.0'.format(channel_select[in_channelselect]))
                    scope.write(':CHANnel{}:SCALe 100E-3'.format(channel_select[in_channelselect]))
                    scope.write(':CHANnel{}:OFFSet 0.0'.format(channel_select[in_channelselect]))
                    scope.write(':TIMebase:SCALe 10E-6')
                    time.sleep(2)
                    scope.write(':MEASure:PERiod? CHANnel{}'.format(channel_select[in_channelselect]))
                    resualt = scope.read()
                    measured_period1 = float(resualt.split(',')[0])
#                     print(measured_period1)
                    scope.write(':MEASure:VPP CHANnel{}'.format(channel_select[in_channelselect]))
                    scope.write(':MEASure:RESults?')
                    result = scope.read()
                    peak_peak_volt1= float(result.split(',')[1])
#                     print(peak_peak_volt1)
                    inst.send_scpi_cmd(':INIT:CONT OFF')
                    inst.send_scpi_cmd(':TRIG:LEV 0.2')
                    inst.send_scpi_cmd(':TRIG:COUPLE ON')#############
                    inst.send_scpi_cmd(':TRIG:CPU:MODE GLOBAL')#############
                    inst.send_scpi_cmd(':TRIG:SOUR:ENAB INT')#############
                    inst.send_scpi_cmd(':TRIG:SEL INT')#############
                    inst.send_scpi_cmd(':TRIG:STAT ON')
                    scope.write(':CHANnel{}:PROBe 1.0'.format(channel_select[in_channelselect]))
                    scope.write(':CHANnel{}:SCALe 100E-3'.format(channel_select[in_channelselect]))
                    scope.write(':CHANnel{}:OFFSet 0.0'.format(channel_select[in_channelselect]))
                    scope.write(':TIMebase:SCALe 10E-6')
                    scope.write(':TRIGger:MODE EDGE')
                    scope.write(':TRIGger:EDGE:SOURce CHANnel{}'.format(channel_select[in_channelselect]))
                    scope.write(':TRIGger:LEVel CHANnel{},120e-3'.format(channel_select[in_channelselect]))
                    time.sleep(5)
                    scope.write(':MEASure:PERiod? CHANnel{}'.format(channel_select[in_channelselect]))
                    resualt = scope.read()
                    measured_period2 = float(resualt.split(',')[0])
#                     print(measured_period2)
                    scope.write(':MEASure:VPP CHANnel{}'.format(channel_select[in_channelselect]))
                    scope.write(':MEASure:RESults?')
                    result = scope.read()
                    peak_peak_volt2= float(result.split(',')[1])
#                     print(peak_peak_volt2)
                    scope.write(':CHANnel{}:DISPlay OFF'.format(channel_select[in_channelselect]))
                    inst.send_scpi_cmd(':INIT:CONT ON')




                # Test for Verticle and Horizontal parameters for decision that was captured on different level
                if abs(peak_peak_volt1-peak_peak_volt2)<120e-3:
#                     print('Difference of peak voltage is {}'.format(peak_peak_volt1-peak_peak_volt2))
                    if abs(1.00e-05-measured_period2)<1.00e-06:
#                         print(1.00e-05-measured_period2)
                        print('\t-->Test  Pass for sampling rate {0}MHz for channel {1} for extnl trigg {2} for Module {3}\n'.format(FreqSampleRate,channel_select[in_channelselect],trigger_select[in_triggerselect],inst_sel))
                    else:
#                         print(1.00e-05-measured_period2)
                        print('\t-->Test  Fail for sampling rate {0}MHz for channel {1} for extnl trigg {2} for Module {3} due to diff in measured period \n'.format(FreqSampleRate,channel_select[in_channelselect],trigger_select[in_triggerselect],inst_sel))
                else:
#                     print(peak_peak_volt1-peak_peak_volt2)
                    print('\t-->Test  Fail for sampling rate {0}MHz for channel {1} for extnl trigg {2} for Module {3} due to diff in peak value \n'.format(FreqSampleRate,channel_select[in_channelselect],trigger_select[in_triggerselect],inst_sel))
                    # Clear the display for next operation
                
                for i in range(4):
                    scope.write(':CHANnel{}:DISPlay OFF'.format(i+1))
                    inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))
                    inst.send_scpi_cmd(':TRIG:STAT OFF')
            
            print('\t***Test completed for sampling rate {}MHz for all the channels of Module {} ***\n'.format(FreqSampleRate,inst_sel))
        if(inst_sel==1):
            print("Please connect all the channels of Module 2 with Scope and Press Enter \n")
            input()
        else:
            print('\t***Test completed for sampling rate {}MHz for all the channels of all Modules ***\n'.format(FreqSampleRate))
    if trigger_select[in_triggerselect] == 'TRG1':
        inst.send_scpi_cmd(':INST:ACT:SEL 1')
        inst.send_scpi_cmd(':TRIG:SOUR:DIS TRG1')
        inst.send_scpi_cmd(':TRIG:SEL TRG1')
        inst.send_scpi_cmd(':TRIG:STAT OFF')
        inst.send_scpi_cmd(':INST:ACT:SEL 2')
        inst.send_scpi_cmd(':TRIG:SOUR:DIS INT')
        inst.send_scpi_cmd(':TRIG:SEL INT')
        inst.send_scpi_cmd(':TRIG:STAT OFF')
        print('\t***Test completed for all sampling rate for trigger input {} ***\n'.format(trigger_select[in_triggerselect]))
        print('\t***Please connect External signal generator with TRG2 input of Proteus')
        print('\t***Please connect all the channels of Module 1 with Scope and press Enter')
        input()
        
    else:
        print('Test Completed for all sampling on each channel for all possible external triggers')

# close inst connection
inst.close_instrument()
disconnect()

# Close the connection to the instrument
scope.close()
resourceManager.close()
            
 
            

***Testing start for Triggering input TRG1***

	***Test begin for sampling rate 1000MHz ***

	-->Test start for sampling rate 1000MHz for channel 1 for extnl trigg TRG1 for Module 1
	-->Test  Pass for sampling rate 1000MHz for channel 1 for extnl trigg TRG1 for Module 1

	-->Test start for sampling rate 1000MHz for channel 2 for extnl trigg TRG1 for Module 1
	-->Test  Fail for sampling rate 1000MHz for channel 2 for extnl trigg TRG1 for Module 1 due to diff in measured period 

	-->Test start for sampling rate 1000MHz for channel 3 for extnl trigg TRG1 for Module 1
	-->Test  Fail for sampling rate 1000MHz for channel 3 for extnl trigg TRG1 for Module 1 due to diff in measured period 

	-->Test start for sampling rate 1000MHz for channel 4 for extnl trigg TRG1 for Module 1


In [ ]:
#disconnect 
dis = input("Press enter to disconnect")
disconnect()
print('Proteus Module is Disconnected')